In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from sklearn.pipeline import Pipeline


In [51]:
df = pd.read_excel(r"C:\Users\oscar\Downloads\Fire Incidents Data.xlsx")

In [37]:
df_2 = pd.read_csv(r"C:\Users\oscar\Downloads\fire-station-locations - 4326.csv")

In [38]:
df_2

,_id,ADDRESS_POINT_ID,ADDRESS_NUMBER,LINEAR_NAME_FULL,ADDRESS,MUNICIPALITY_NAME,CENTRELINE_ID,OBJECTID,ID,STATION,YEAR_BUILD,WARD,WARD_NAME,TYPE_DESC,PUBLIC_ED_OFFICE,FIRE_PREV_OFFICE,FIRE_OTHER,geometry
0,1,4236992,900,Tapscott Rd,900 Tapscott Rd,Scarborough,4236991,1520443,74,211,1982,23,Scarborough North,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.24..."
1,2,308265,351,Birchmount Rd,351 Birchmount Rd,Scarborough,112783,1527311,54,0,1925,20,Scarborough Southwest,Admin,NaN,Fire Prevention Office (E),Fire Museum,"{'type': 'MultiPoint', 'coordinates': [[-79.26..."
2,3,8155506,15,Rotherham Ave,15 Rotherham Ave,York,8155517,1541445,99,0,1985,5,York South-Weston,Admin,NaN,NaN,Warehouse and Mechanical Building,"{'type': 'MultiPoint', 'coordinates': [[-79.47..."
3,4,764237,106,Ascot Ave,106 Ascot Ave,former Toronto,1140634,1541526,12,342,1915,9,Davenport,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.44..."
4,5,6349868,745,Meadowvale Rd,745 Meadowvale Rd,Scarborough,6349869,1567567,73,214,1982,25,Scarborough-Rouge Park,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,12763904,260,Adelaide St W,260 Adelaide St W,former Toronto,12763900,4615182,25,332,1971,10,Spadina-Fort York,Fire Station,NaN,Fire Prevention Office (S),Division Commander / Platoon Chiefs,"{'type': 'MultiPoint', 'coordinates': [[-79.38..."
97,98,30055401,2945,Keele St,2945 Keele St,North York,30110004,4635214,49,144,0,6,York Centre,Fire Station,NaN,NaN,NaN,"{'type': 'MultiPoint', 'coordinates': [[-79.48..."
98,99,6044964,947,Martin Grove Rd,947 Martin Grove Rd,Etobicoke,6044958,4653861,6,441,1964,1,Etobicoke North,Fire Station,NaN,NaN,West Training Centre / Fire Training Tower,"{'type': 'MultiPoint', 'coordinates': [[-79.57..."
99,100,19832,256,Cosburn Ave,256 Cosburn Ave,East York,14646506,4659390,45,322,1994,14,Toronto-Danforth,Fire Station,NaN,NaN,Accreditation,"{'type': 'MultiPoint', 'coordinates': [[-79.34..."


In [52]:
df

,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties
0,1875241,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,"7 - Fully involved (total structure, vehicle, ...",2018-02-24T21:04:29,2018-02-24T21:10:11,0.0
1,1875242,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,NaN,NaN,NaN,NaN,NaN,NaN,2 - Fire with no evidence from street,2018-02-24T21:24:43,2018-02-24T21:29:31,0.0
2,1875243,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-25T13:29:59,2018-02-25T13:36:49,0.0
3,1875244,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,98 - Not applicable: Alarm operated OR presenc...,9 - Type undetermined,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,2018-02-25T14:13:39,2018-02-25T14:18:07,0.0
4,1875245,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-25T18:20:43,2018-02-25T18:26:19,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25855,1901096,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-24T07:28:16,2021-06-24T07:34:12,0.0
25856,1901097,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,5000.0,0.0,NaN,2021-06-24T08:08:15,...,98 - Not applicable: Alarm operated OR presenc...,2 - Hardwired (standalone),7 - Not applicable: Occupant(s) first alerted ...,"7 - Spread to other floors, confined to building",8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2021-06-24T07:59:40,2021-06-24T08:06:34,0.0
25857,1901098,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,20000.0,1.0,NaN,2021-06-24T11:50:49,...,98 - Not applicable: Alarm operated OR presenc...,9 - Type undetermined,9 - Undetermined,3 - Spread to entire room of origin,1 - Sprinkler system activated,1 - Full sprinkler system present,1 - Fire extinguished prior to arrival,2021-06-24T11:45:44,2021-06-24T11:48:49,0.0
25858,1901099,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-24T14:04:07,2021-06-24T14:08:55,0.0


In [54]:
df['Possible_Cause'].unique

<bound method Series.unique of 0                                     99 - Undetermined
1                              03 - Suspected Vandalism
2                                                   NaN
3                                     99 - Undetermined
4                                                   NaN
                              ...                      
25855                                               NaN
25856    60 - Other unintentional cause, not classified
25857                                   44 - Unattended
25858                                               NaN
25859         11 - Children Playing (Ages 11 and under)
Name: Possible_Cause, Length: 25860, dtype: object>

In [56]:
df['Possible_Cause'].unique()

array(['99 - Undetermined', '03 - Suspected Vandalism', nan,
       '20 - Design/Construction/Installation/Maintenance Deficiency',
       '52 - Electrical Failure',
       '46 - Used or Placed too close to combustibles', '44 - Unattended',
       '28 - Routine maintenance deficiency, eg creosote, lint, grease buildup',
       '98 - Unintentional, cause undetermined', '80 - Exposure fire',
       '60 - Other unintentional cause, not classified',
       '51 - Mechanical Failure',
       '47 - Improper handling of ignition source or ignited material',
       '45 - Improperly Discarded', '01 - Suspected Arson',
       '50 - Other misuse of ignition source/material ignited',
       '49 - Improper Storage', '48 - Used for purpose not intended',
       '11 - Children Playing (Ages 11 and under)',
       '12 - Vehicle Accident/Collision',
       '04 - Suspected Youth Vandalism (Ages 12 to 17)',
       '990 - Under Investigation', '73 - Natural Cause',
       '02 - Riot/Civil Commotion', '72 -

In [39]:
import pandas as pd


# Define the default city and country values
city = "Toronto"
country = "Canada"

# Update the addresses by adding the city and country
df_2['ADDRESS'] = df_2['ADDRESS'] + f", {city}, {country}"

In [41]:
df_2['ADDRESS']

0          900 Tapscott Rd, Toronto, Canada
1        351 Birchmount Rd, Toronto, Canada
2         15 Rotherham Ave, Toronto, Canada
3            106 Ascot Ave, Toronto, Canada
4        745 Meadowvale Rd, Toronto, Canada
                       ...                 
96       260 Adelaide St W, Toronto, Canada
97           2945 Keele St, Toronto, Canada
98     947 Martin Grove Rd, Toronto, Canada
99         256 Cosburn Ave, Toronto, Canada
100    40 Queen's Plate Dr, Toronto, Canada
Name: ADDRESS, Length: 101, dtype: object

In [42]:
from geopy.geocoders import Nominatim
import pandas as pd

# Initialize a geocoder with your user agent
geolocator = Nominatim(user_agent="my-custom-application")

# Define a function to get zip codes based on addresses
def get_zip_code(address):
    location = geolocator.geocode(address, exactly_one=True)
    if location and location.raw.get("address"):
        return location.raw["address"].get("postcode", "N/A")
    else:
        return "N/A"

# Apply the function to your dataset to create a new 'ZipCode' column
df_2['ZipCode'] = df_2['ADDRESS'].apply(get_zip_code)

# Save the updated dataset with zip codes to a new CSV file
# data.to_csv('dataset_with_zip_codes.csv', index=False)


In [43]:
df_2['ZipCode']

0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
96     N/A
97     N/A
98     N/A
99     N/A
100    N/A
Name: ZipCode, Length: 101, dtype: object

In [46]:

# import module 
from geopy.geocoders import Nominatim 
  
# initialize Nominatim API  
geolocator = Nominatim(user_agent="geoapiExercises") 
  
# place input by geek 
place = "Boring road patna"
location = geolocator.geocode(place) 
  
# traverse the data 
data = location.raw 
loc_data = data['display_name'].split() 
print("Full Location") 
print(loc_data) 
print("Zip code : ",loc_data[-2])

GeocoderInsufficientPrivileges: Non-successful status code 403

In [47]:
import requests

# Replace 'YOUR_API_KEY' with your actual Google Maps API key
api_key = 'YOUR_API_KEY'

# Define the base URL for the Google Maps Geocoding API
base_url = 'https://maps.googleapis.com/maps/api/geocode/json'

# Function to get the zip code from an address
def get_zip_code(address):
    params = {
        'address': address,
        'key': api_key,
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract the zip code from the first result
            result = data['results'][0]
            for component in result['address_components']:
                if 'postal_code' in component['types']:
                    return component['short_name']
    return 'N/A'

# Test the function
address = "900 Tapscott Rd, Toronto, Canada"
zip_code = get_zip_code(address)
print("Zip Code:", zip_code)


Zip Code: N/A


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25860 entries, 0 to 25859
Data columns (total 43 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   _id                                                            25860 non-null  int64  
 1   Area_of_Origin                                                 20065 non-null  object 
 2   Building_Status                                                14285 non-null  object 
 3   Business_Impact                                                14283 non-null  object 
 4   Civilian_Casualties                                            25638 non-null  float64
 5   Count_of_Persons_Rescued                                       25637 non-null  float64
 6   Estimated_Dollar_Loss                                          20079 non-null  float64
 7   Estimated_Number_Of_Persons_Displaced                     

In [6]:
df.columns

Index(['_id', 'Area_of_Origin', 'Building_Status', 'Business_Impact',
       'Civilian_Casualties', 'Count_of_Persons_Rescued',
       'Estimated_Dollar_Loss', 'Estimated_Number_Of_Persons_Displaced',
       'Exposures', 'Ext_agent_app_or_defer_time', 'Extent_Of_Fire',
       'Final_Incident_Type', 'Fire_Alarm_System_Impact_on_Evacuation',
       'Fire_Alarm_System_Operation', 'Fire_Alarm_System_Presence',
       'Fire_Under_Control_Time', 'Ignition_Source', 'Incident_Number',
       'Incident_Station_Area', 'Incident_Ward', 'Initial_CAD_Event_Type',
       'Intersection', 'Last_TFS_Unit_Clear_Time', 'Latitude',
       'Level_Of_Origin', 'Longitude', 'Material_First_Ignited',
       'Method_Of_Fire_Control', 'Number_of_responding_apparatus',
       'Number_of_responding_personnel', 'Possible_Cause', 'Property_Use',
       'Smoke_Alarm_at_Fire_Origin',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Failure',
       'Smoke_Alarm_at_Fire_Origin_Alarm_Type',
       'Smoke_Alarm_Impact_on_Persons

In [8]:
pip install geopy

                                              0.0/125.4 kB ? eta -:--:--
     -------------------------------------- 125.4/125.4 kB 7.7 MB/s eta 0:00:00
                                              0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from geopy.geocoders import Nominatim

# Initialize a geocoder
geolocator = Nominatim(user_agent="myGeocoder")

# Define a function to get postal code based on latitude and longitude
def get_postal_code(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location and location.raw.get("address"):
        return location.raw["address"].get("postcode", "N/A")
    else:
        return "N/A"

# Assuming you have a Pandas DataFrame named 'df' with 'latitude' and 'longitude' columns
df["postal_code"] = df.apply(lambda row: get_postal_code(row["Latitude"], row["Longitude"]), axis=1)


GeocoderInsufficientPrivileges: Non-successful status code 403

In [11]:
from geopy.geocoders import Nominatim
import pandas as pd

# Initialize a geocoder with your user agent
geolocator = Nominatim(user_agent="MyGeocodingApp/1.0")

# Define a function to get postal code based on latitude and longitude
def get_postal_code(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location and location.raw.get("address"):
        return location.raw["address"].get("postcode", "N/A")
    else:
        return "N/A"

# Assuming you have a Pandas DataFrame named 'df' with 'latitude' and 'longitude' columns
# df should look like:
#   latitude  longitude
# 0   12.9716    77.5946
# 1   40.7128   -74.0060
df["postal_code"] = df.apply(lambda row: get_postal_code(row["Latitude"], row["Longitude"]), axis=1)

# Save the DataFrame with postal codes to a CSV file
df.to_excel(r"C:\Users\oscar\Downloads\Fire Incidents Data2.xlsx", index=False)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=43.68891266&lon=-79.47685184&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [25]:
# Save the DataFrame with postal codes to a CSV file
df.to_excel(r"C:\Users\oscar\Downloads\Fire Incidents Data2.xlsx", index=False)

In [32]:
count_distinct = len(df['Incident_Ward'].unique())
print(count_distinct)


46


In [35]:
df['Incident_Ward'] == '0.'


0        False
1        False
2        False
3        False
4        False
         ...  
25855    False
25856    False
25857    False
25858    False
25859    False
Name: Incident_Ward, Length: 25860, dtype: bool

In [36]:
df

,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties,POSTAL_CODE,city
0,1875241,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,NaN,NaN,NaN,NaN,"7 - Fully involved (total structure, vehicle, ...",2018-02-24T21:04:29,2018-02-24T21:10:11,0.0,N/A,Toronto
1,1875242,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,NaN,NaN,NaN,NaN,2 - Fire with no evidence from street,2018-02-24T21:24:43,2018-02-24T21:29:31,0.0,N/A,Toronto
2,1875243,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-02-25T13:29:59,2018-02-25T13:36:49,0.0,N/A,Toronto
3,1875244,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,2018-02-25T14:13:39,2018-02-25T14:18:07,0.0,N/A,Toronto
4,1875245,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-02-25T18:20:43,2018-02-25T18:26:19,0.0,N/A,Toronto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25855,1901096,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-06-24T07:28:16,2021-06-24T07:34:12,0.0,N/A,Toronto
25856,1901097,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,5000.0,0.0,NaN,2021-06-24T08:08:15,...,7 - Not applicable: Occupant(s) first alerted ...,"7 - Spread to other floors, confined to building",8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2021-06-24T07:59:40,2021-06-24T08:06:34,0.0,N/A,Toronto
25857,1901098,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,20000.0,1.0,NaN,2021-06-24T11:50:49,...,9 - Undetermined,3 - Spread to entire room of origin,1 - Sprinkler system activated,1 - Full sprinkler system present,1 - Fire extinguished prior to arrival,2021-06-24T11:45:44,2021-06-24T11:48:49,0.0,N/A,Toronto
25858,1901099,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-06-24T14:04:07,2021-06-24T14:08:55,0.0,N/A,Toronto


In [ ]:
dollor_loss= df['Estimated_Dollar_Loss'].to_numpy()

size = np.clip(10 * dollor_loss / np.mean(dollor_loss),1.0,np.max(dollor_loss))
patches = []

for polygon in areas:
    polygon_patch = Polygon(polygon, True)
    patches.append(polygon_patch)


colors = 100*np.random.rand(len(patches))
p = PatchCollection(patches, alpha=0.4,edgecolor = 'k')
fig, ax = plt.subplots(figsize=(15, 10))
p.set_array(np.array(colors))
ax.add_collection(p)
ax.set_title('Fire incidents in Toronto. Each circle indicates an incident with size of \
the circle scaled by estimated loss. Maximum loss:%.2f.\nBlue squares show fire station locations within each area'%dollor_loss.max())
plt.autoscale(tight=True)
plt.scatter(df['Longitude'], df['Latitude'] , s=size, c ='r', alpha=0.3, marker = 'o')
plt.scatter(stations[:,0], stations[:,1] , marker = 's', s = 10, facecolors='none', edgecolors='b')

plt.show()

In [24]:
df.to_excel("data_with_postal_codes.xlsx", index=False)

In [21]:
df['city'] = 'Toronto'

In [22]:
df

,_id,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Exposures,Ext_agent_app_or_defer_time,...,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time,TFS_Firefighter_Casualties,POSTAL_CODE,city
0,1875241,81 - Engine Area,NaN,NaN,0.0,0.0,15000.0,NaN,NaN,2018-02-24T21:12:00,...,NaN,NaN,NaN,NaN,"7 - Fully involved (total structure, vehicle, ...",2018-02-24T21:04:29,2018-02-24T21:10:11,0.0,N/A,Toronto
1,1875242,"75 - Trash, rubbish area (outside)",NaN,NaN,0.0,0.0,50.0,NaN,NaN,2018-02-24T21:29:42,...,NaN,NaN,NaN,NaN,2 - Fire with no evidence from street,2018-02-24T21:24:43,2018-02-24T21:29:31,0.0,N/A,Toronto
2,1875243,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-02-25T13:29:59,2018-02-25T13:36:49,0.0,N/A,Toronto
3,1875244,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,NaN,2018-02-25T14:19:25,...,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,2018-02-25T14:13:39,2018-02-25T14:18:07,0.0,N/A,Toronto
4,1875245,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-02-25T18:20:43,2018-02-25T18:26:19,0.0,N/A,Toronto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25855,1901096,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-06-24T07:28:16,2021-06-24T07:34:12,0.0,N/A,Toronto
25856,1901097,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,5000.0,0.0,NaN,2021-06-24T08:08:15,...,7 - Not applicable: Occupant(s) first alerted ...,"7 - Spread to other floors, confined to building",8 - Not applicable - no sprinkler system present,3 - No sprinkler system,1 - Fire extinguished prior to arrival,2021-06-24T07:59:40,2021-06-24T08:06:34,0.0,N/A,Toronto
25857,1901098,24 - Cooking Area or Kitchen,01 - Normal (no change),8 - Not applicable (not a business),0.0,0.0,20000.0,1.0,NaN,2021-06-24T11:50:49,...,9 - Undetermined,3 - Spread to entire room of origin,1 - Sprinkler system activated,1 - Full sprinkler system present,1 - Fire extinguished prior to arrival,2021-06-24T11:45:44,2021-06-24T11:48:49,0.0,N/A,Toronto
25858,1901099,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-06-24T14:04:07,2021-06-24T14:08:55,0.0,N/A,Toronto


In [4]:
import geopy as gy
import pandas as pd


def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']


geolocator = gy.Nominatim(user_agent='my-application')

df['zipcodes'] = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

ConfigurationError: Using Nominatim with default or sample `user_agent` "my-application" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.

In [6]:
import geopy as gy
import pandas as pd

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']

geolocator = gy.Nominatim(user_agent='my-custom-application')

df['zipcodes'] = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')


KeyError: 'postcode'

In [9]:
import geopy
import pandas as pd


geolocator = geopy.Nominatim(user_agent="check_1")

def get_zip_code(x):
    location = geolocator.reverse("{}, {}".format(x['Latitude'],x['Longitude']))
    return location.raw['address']['postcode']
df['zipcode'] = df.apply(lambda x: get_zip_code(x), axis = 1)
print(df.head())

KeyError: 'postcode'

In [10]:
import geopy
import pandas as pd

geolocator = geopy.Nominatim(user_agent="check_1")

def get_zip_code(x):
    location = geolocator.reverse("{}, {}".format(x['Latitude'], x['Longitude']))
    if 'postcode' in location.raw['address']:
        return location.raw['address']['postcode']
    else:
        return "N/A"

df['zipcode'] = df.apply(lambda x: get_zip_code(x), axis=1)
print(df.head())


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=43.69471&lon=-79.50638&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000192FC699C10>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))

In [11]:
df_zip = pd.read_csv(r"C:\Users\oscar\Downloads\CanadianPostalCodes202203.csv")

In [12]:
df_zip

,POSTAL_CODE,CITY,PROVINCE_ABBR,TIME_ZONE,LATITUDE,LONGITUDE
0,L8E 2L1,STONEY CREEK,ON,5,43.229972,-79.729382
1,G6P 7J8,VICTORIAVILLE,QC,5,46.041090,-71.931014
2,H7G 3H1,LAVAL,QC,5,45.576954,-73.695084
3,T5Z 3L7,EDMONTON,AB,7,53.627949,-113.486899
4,K0H 2V0,TICHBORNE,ON,5,44.672522,-76.672489
...,...,...,...,...,...,...
895310,S7S 1K1,SASKATOON,SK,6,52.161127,-106.594685
895311,J1S 1R8,WINDSOR,QC,5,45.556139,-71.997663
895312,V2G 4Y2,WILLIAMS LAKE,BC,8,52.100324,-122.184050
895313,K6V 2Y7,BROCKVILLE,ON,5,44.597185,-75.672561


In [14]:
df_zip['Latitude'] = df_zip['LATITUDE']

In [15]:
df_zip['Longitude'] = df_zip['LONGITUDE']

In [16]:
df_zip

,POSTAL_CODE,CITY,PROVINCE_ABBR,TIME_ZONE,LATITUDE,LONGITUDE,Latitude,Longitude
0,L8E 2L1,STONEY CREEK,ON,5,43.229972,-79.729382,43.229972,-79.729382
1,G6P 7J8,VICTORIAVILLE,QC,5,46.041090,-71.931014,46.041090,-71.931014
2,H7G 3H1,LAVAL,QC,5,45.576954,-73.695084,45.576954,-73.695084
3,T5Z 3L7,EDMONTON,AB,7,53.627949,-113.486899,53.627949,-113.486899
4,K0H 2V0,TICHBORNE,ON,5,44.672522,-76.672489,44.672522,-76.672489
...,...,...,...,...,...,...,...,...
895310,S7S 1K1,SASKATOON,SK,6,52.161127,-106.594685,52.161127,-106.594685
895311,J1S 1R8,WINDSOR,QC,5,45.556139,-71.997663,45.556139,-71.997663
895312,V2G 4Y2,WILLIAMS LAKE,BC,8,52.100324,-122.184050,52.100324,-122.184050
895313,K6V 2Y7,BROCKVILLE,ON,5,44.597185,-75.672561,44.597185,-75.672561


In [17]:
# Create a dictionary mapping (lat, long) to zip code from df2
lat_long_to_zip = df_zip.set_index(['Latitude', 'Longitude'])['POSTAL_CODE'].to_dict()

# Add a new 'ZipCode' column to df1 based on the mapping
df['POSTAL_CODE'] = df.apply(lambda row: lat_long_to_zip.get((row['Latitude'], row['Longitude']), 'N/A'), axis=1)

In [20]:
df['POSTAL_CODE']

0        N/A
1        N/A
2        N/A
3        N/A
4        N/A
        ... 
25855    N/A
25856    N/A
25857    N/A
25858    N/A
25859    N/A
Name: POSTAL_CODE, Length: 25860, dtype: object